# Artificial Intelligence Spring 2019, Lab 7

# This program introduces the following concepts:

*		a) Reading a stream of images from a webcamera, and displaying the video (learned in lab 6)
*		b) Skin color detection (learned in lab 6)
*		c) Background differencing
*		d) Visualizing motion history


In [177]:
import numpy as np
import cv2
from collections import deque


# skin color detection

In [178]:
# Function that detects whether a pixel belongs to the skin based on RGB values
# src - the source color image
def mySkinDetect(src):
    # Surveys of skin color modeling and detection techniques:
    # 1. Vezhnevets, Vladimir, Vassili Sazonov, and Alla Andreeva. "A survey on pixel-based skin color detection techniques." Proc. Graphicon. Vol. 3. 2003.
    # 2. Kakumanu, Praveen, Sokratis Makrogiannis, and Nikolaos Bourbakis. "A survey of skin-color modeling and detection methods." Pattern recognition 40.3 (2007): 1106-1122.
    dst = np.zeros((src.shape[0], src.shape[1], 1), dtype = "uint8")
    for i in range(src.shape[0]):
        for j in range(src.shape[1]):
            #b,g,r = src[i,j]
            b = int(src[i,j][0])
            g = int(src[i,j][1])
            r = int(src[i,j][2])
            if(r>45 and g>20 and b>5 and max(r,g,b)-min(r,g,b)>15 and abs(r-g)>15 and r>g and r>b):
                dst[i,j] = 255
    return dst

# frame-to-frame differencing

In [179]:
# Function that does frame differencing between the current frame and the previous frame
# prev - the previous color image
# curr - the current color image
# dst - the destination grayscale image where pixels are colored white if the corresponding pixel intensities in the current
# and previous image are not the same
def myFrameDifferencing(prev, curr):
    # For more information on operation with arrays: 
    # http://docs.opencv.org/modules/core/doc/operations_on_arrays.html
    
    
    
    #dst = np.zeros((prev.shape[0], prev.shape[1], 1), dtype = "uint8")
    dst = cv2.absdiff(prev,curr)
    dst = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY) #convert color image to gray scale image
    _, dst = cv2.threshold(dst, 50, 255, cv2.THRESH_BINARY) #if pixel is greater than 50, set the pixel to white (only get the second value)
                
            
    return dst

# motion energy templates
* example 1: the bottom row displays a cumulative binary motion energy image sequence corresponding to the frames above
![title](mh1.png)
* example 2: pixel intensity is a function of the motion history at that location, where brighter values correspond to more recent motion, three actions: sit-down, arms-raise, crouch-down
![title](mh2.png)

In [180]:
# Function that accumulates the frame differences for a certain number of pairs of frames
# mh - vector of frame difference images
# dst - the destination grayscale image to store the accumulation of the frame difference images
def myMotionEnergy(mh):
    # the window of time is 3
    mh0 = mh[0]
    mh1 = mh[1]
    mh2 = mh[2]
    dst = np.zeros((mh0.shape[0], mh0.shape[1], 1), dtype = "uint8")
    for i in range(mh0.shape[0]):
        for j in range(mh0.shape[1]):
            if mh0[i,j] == 255 or mh1[i,j] == 255 or mh2[i,j] == 255:
                dst[i,j] = 255
    return dst

In [181]:
def manualtemplatematch(curr):
    img = myMotionEnergy(curr)
    template = cv2.imread('template.png',0)
    ret,dst = cv2.threshold(template,127,255,cv2.THRESH_BINARY)
    templateresized = cv2.resize(dst, (curr.shape[0], curr.shape[1]))
    n = 0
    #print(img.shape)
    #print(templateresized.shape)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if img[i,j] == templateresized[i,j]:
                n +=1
    if n/22500 >=.0065:
        return True
    return False
            

In [182]:
def segment(src, threshold = 50):
    global bg 
    diff = cv2.absdiff(bg.astype("uint8"),src)
    thresholded = cv2.threshold(diff,threshold,255,cv2.THRESH_BINARY)[1]
    (_,cnts, _) = cv2.findContours(mySkinDetect(src), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(cnts)==0:
        return
    else:
        segmented = max(cnts, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(segmented)
        cv2.rectangle(src, (x, y), (x+w, y+h), (0, 255, 0), 2)
        return (cv2.drawContours(src, segmented, -1, (255, 255, 0), 1))
        #return (thresholded, segmented)
    

In [183]:
def boundingBox(curr):
    
    (_,cnts, _) = cv2.findContours(mySkinDetect(curr), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(cnts)==0:
        return
    else:
        segmented = max(cnts, key=cv2.contourArea)
        return segmented


    

In [184]:
def count(curr, segmented):
    #fnd the convex hull of the segmented hand region
    chull = cv2.convexHull(segmented)
    thresholded = mySkinDetect(curr)
    # find the most extreme points in the convex hull
    extreme_top    = tuple(chull[chull[:, :, 1].argmin()][0])
    extreme_bottom = tuple(chull[chull[:, :, 1].argmax()][0])
    extreme_left   = tuple(chull[chull[:, :, 0].argmin()][0])
    extreme_right  = tuple(chull[chull[:, :, 0].argmax()][0])
    

    # find the center of the palm
    cX = (extreme_left[0] + extreme_right[0]) / 2
    cY = (extreme_top[1] + extreme_bottom[1]) / 2
    dist1 = ((cX-extreme_left[0])**2 + (cY-extreme_left[1])**2)**0.5
    dist2 = ((cX-extreme_right[0])**2 + (cY-extreme_right[1])**2)**0.5
    dist3 = ((cX-extreme_top[0])**2 + (cY-extreme_top[1])**2)**0.5
    dist4 = ((cX-extreme_bottom[0])**2 + (cY-extreme_bottom[1])**2)**0.5
    # find the maximum euclidean distance between the center of the palm
    # and the most extreme points of the convex hull
    x = np.array([(cX,cY)])
    y = np.array([extreme_left, extreme_right, extreme_top, extreme_bottom])
    
    distance = np.array ([[dist1], [dist2], [dist3], [dist4]])[0]
    maximum_distance = distance[distance.argmax()]
    
    # calculate the radius of the circle with 80% of the max euclidean distance obtained
    radius = int(0.8 * maximum_distance)
    
    # find the circumference of the circle
    circumference = (2 * np.pi * radius)

    # take out the circular region of interest which has 
# the palm and the fingers
    circular_roi = np.zeros(thresholded.shape[:2], dtype="uint8")
    # draw the circular ROI
    cv2.circle(circular_roi, (int(cX), int(cY)), int(radius), 255, 1)
    

	# take bit-wise AND between thresholded hand using the circular ROI as the mask
	# which gives the cuts obtained using mask on the thresholded hand image
    circular_roi = cv2.bitwise_and(thresholded, thresholded, mask=circular_roi)

	# compute the contours in the circular ROI
    (_, cnts, _) = cv2.findContours(circular_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

	# initalize the finger count
    count = 0

	# loop through the contours found
    for c in cnts:
		# compute the bounding box of the contour
        (x, y, w, h) = cv2.boundingRect(c)

		# increment the count of fingers only if -
		# 1. The contour region is not the wrist (bottom area)
		# 2. The number of points along the contour does not exceed
		#     25% of the circumference of the circular ROI
        if ((cY + (cY * 0.25)) > (y + h)) and ((circumference * 0.25) > c.shape[0]):
            count += 1
    if (manualtemplatematch(curr)) == True:
        return ("Waving")
    else:
        if count == 2: 
            return ("Scissor")
        if count >=4 :
            return ("Paper")
        if count <=1:
            return ("Rock")
    return ("No Gesture Detected")

In [186]:
def main():
    # a) Reading a stream of images from a webcamera, and displaying the video
    # open the video camera no. 0/modules/highgui/doc/reading_and_writing_images_and_video.html
    cap = cv2.VideoCapture(0)
    
    #if not successful, exit program
    if not cap.isOpened():
        print("Cannot open the video cam")
        return -1

    # read a new frame from video
    success, prev_frame = cap.read()
    
    #if not successful, exit program
    if not success:
        print("Cannot read a frame from video stream")
        return -1
    cv2.namedWindow("frame", cv2.WINDOW_AUTOSIZE)
    
    prev_frame = cv2.resize(prev_frame,(150,150))
    fMH1 = np.zeros((prev_frame.shape[0], prev_frame.shape[1], 1), dtype = "uint8")
    fMH2 = fMH1.copy()
    fMH3 = fMH1.copy()
    myMotionHistory = deque([fMH1, fMH2, fMH3])

    
    while(True):
        #read a new frame from video
        success, curr_frame = cap.read()
        curr_frame = cv2.resize(curr_frame,(150,150))
        
        if not success:
            print("Cannot read a frame from video stream")
            break
        curr_frame = cv2.flip(curr_frame,1)
  

        cv2.imshow('frame',curr_frame)

        # b) Skin color detection
        mySkin = mySkinDetect(curr_frame)
        cv2.imshow('mySkinDetect',mySkin)

        #Background differencing
        #bd = backgroundDifferencing(curr_frame)
        #cv2.imshow('myBackgroundDifferencing', bd)
        
        # c) Frame by frame differencing
        #frameDest = myFrameDifferencing(prev_frame, curr_frame)
        #cv2.imshow('myFrameDifferencing',frameDest)
        
        #myMotionHistory.popleft()
        #myMotionHistory.append(frameDest)
        #myMH = myMotionEnergy(myMotionHistory)
        #cv2.imshow('myMotionHistory',myMH)

        prev_frame = curr_frame
        
         #template matching
        #tm = templateMatching2(curr_frame)
        #cv2.imshow('templateMatching', tm)
        
        #bounding box
        segmented = boundingBox(curr_frame)
        x, y, w, h = cv2.boundingRect(segmented)
        bound = cv2.rectangle(curr_frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        bound =(cv2.drawContours(curr_frame, segmented, -1, (255, 255, 0), 1))
        
        
        fingers = count(curr_frame, segmented)
     
        bound = cv2.putText(curr_frame, str(fingers), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, .5, (0,0,255), 2)
        cv2.imshow('myBoundingBox', bound)
        
        
        
        # wait for 'q' key press. If 'q' key is pressed, break loop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    return 0

In [187]:
if __name__ == "__main__":
    main()

error: OpenCV(3.4.1) C:\Miniconda3\conda-bld\opencv-suite_1533128839831\work\modules\imgproc\src\convhull.cpp:137: error: (-215) total >= 0 && (depth == 5 || depth == 4) in function cv::convexHull
